In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import Data_v2
import Data_analysis_func
import Model_checks
import seaborn as sns
from scipy.stats import multivariate_normal
from sklearn.linear_model import LinearRegression
import Outlier_detection_v2
import Genes_self_similarity_v2
from matplotlib_venn import venn2, venn2_circles
import Feature_engineering
import Clusters

In [2]:
data_norm_flagged = Data_v2.get_norm_flagged_data()

/Users/samsongourevitch/Documents/Chlamy_Project_Github/Chlamy_project/Data_v2.py:185: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_copy.at[index, column_name] -= wt_mean_y2[i - 1]
/Users/samsongourevitch/Documents/Chlamy_Project_Github/Chlamy_project/Data_v2.py:193: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_copy.at[index, column_name] -= wt_mean_ynpq[i - 1]


In [3]:
alpha = 0.05
Outlier_detection_v2.detect_outlier_mean_reduced_v2(data_norm_flagged, alpha)
Outlier_detection_v2.detect_outliers_fv_fm_ynpq_end(data_norm_flagged, alpha)
Outlier_detection_v2.detect_diverging_outliers(data_norm_flagged, alpha, w=10, p=(1/2))

/Users/samsongourevitch/Documents/Chlamy_Project_Github/Chlamy_project/Outlier_detection_v2.py:379: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Fal

In [4]:
outlier_types = data_norm_flagged.columns[data_norm_flagged.columns.str.startswith('outlier_')]

for outlier_type in outlier_types:
    data_norm_flagged[outlier_type + '_count'] = data_norm_flagged.groupby(['mutated_genes', 'light_regime'])[outlier_type].transform('sum')

In [5]:
data_norm_ok_y2 = data_norm_flagged[data_norm_flagged['flag_y2'] == 'ok']
data_norm_ok_ynpq = data_norm_flagged[data_norm_flagged['flag_ynpq'] == 'ok']

outlier_types_y2 = data_norm_flagged.columns[(data_norm_ok_y2.columns.str.startswith('outlier_')) & (data_norm_ok_y2.columns.str.endswith('y2'))]
outlier_types_ynpq = data_norm_flagged.columns[(data_norm_ok_ynpq.columns.str.startswith('outlier_')) & (data_norm_ok_ynpq.columns.str.endswith('ynpq'))]

for outlier_type in outlier_types:
    data_norm_ok_y2[outlier_type + '_count'] = data_norm_ok_y2.groupby(['mutated_genes', 'light_regime'])[outlier_type].transform('sum')
    data_norm_ok_ynpq[outlier_type + '_count'] = data_norm_ok_ynpq.groupby(['mutated_genes', 'light_regime'])[outlier_type].transform('sum')

/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/333543713.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_norm_ok_y2[outlier_type + '_count'] = data_norm_ok_y2.groupby(['mutated_genes', 'light_regime'])[outlier_type].transform('sum')
/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/333543713.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_norm_ok_ynpq[outlier_type + '_count'] = data_norm_ok_ynpq.groupby(['mutated_genes', 'light_regime'])[outlier_type].transfo

In [6]:
priority_mutants_y2 = data_norm_ok_y2[data_norm_ok_y2['mutated_genes_light_regime_count'] >= 2]
priority_mutants_ynpq = data_norm_ok_ynpq[data_norm_ok_ynpq['mutated_genes_light_regime_count'] >= 2]

fluctuating_light_regimes = ['30s-30s', '1min-1min', '10min-10min', '2h-2h']
continuous_light_regimes = ['20h_ML', '20h_HL']

def filter_mutants(group):
    # Check if any outlier type has at least 2 True counts for fluctuating light regimes
    has_at_least_2_true_fluctuating = np.any(group.loc[group['light_regime'].isin(['30s-30s', '1min-1min', '10min-10min', '2h-2h']), outlier_types_y2 + '_count'] >= 2)

    has_at_least_2_true_continuous = np.any(group.loc[group['light_regime'].isin(['20h_ML', '20h_HL']), outlier_types_y2 + '_count'] >= 2)

    is_one_of_the_true_fluctuating = np.any(group.loc[group['light_regime'].isin(fluctuating_light_regimes), outlier_types_y2] == True)

    is_one_of_the_true_continuous = np.any(group.loc[group['light_regime'].isin(continuous_light_regimes), outlier_types_y2] == True)
    
    # Check if all outlier types have 0 True counts for continuous light regimes
    has_all_zero_continuous = np.all(group.loc[group['light_regime'].isin(['20h_ML', '20h_HL']), outlier_types_y2 + '_count'] == 0)

    has_all_zero_fluctuating = np.all(group.loc[group['light_regime'].isin(['30s-30s', '1min-1min', '10min-10min', '2h-2h']), outlier_types_y2 + '_count'] == 0)

    # Check if the group has at least of the continuous light regimes
    has_at_least_one_continuous = np.any(group['light_regime'].isin(['20h_ML', '20h_HL']))

    has_at_least_one_fluctuating = np.any(group['light_regime'].isin(['30s-30s', '1min-1min', '10min-10min', '2h-2h']))
    
    return (has_at_least_2_true_fluctuating and has_all_zero_continuous and has_at_least_one_continuous and is_one_of_the_true_fluctuating) | (has_at_least_2_true_continuous and has_all_zero_fluctuating and has_at_least_one_fluctuating and is_one_of_the_true_continuous)

# Apply the filter function to each group defined by 'mutated_genes'
filtered_genes_y2 = priority_mutants_y2.groupby('mutated_genes').apply(filter_mutants)
selected_genes_ids_y2 = filtered_genes_y2[filtered_genes_y2].index.get_level_values('mutated_genes').tolist()
priority_mutants_y2_filtered = priority_mutants_y2[priority_mutants_y2['mutated_genes'].isin(selected_genes_ids_y2)]

filtered_genes_ynpq = priority_mutants_ynpq.groupby('mutated_genes').apply(filter_mutants)
selected_genes_ids_ynpq = filtered_genes_ynpq[filtered_genes_ynpq].index.get_level_values('mutated_genes').tolist()
priority_mutants_ynpq_filtered = priority_mutants_ynpq[priority_mutants_ynpq['mutated_genes'].isin(selected_genes_ids_ynpq)]

/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/2106697792.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_genes_y2 = priority_mutants_y2.groupby('mutated_genes').apply(filter_mutants)
/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/2106697792.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_genes_ynpq = priority_mutants_ynpq.groupby('mutated_genes'

In [7]:
priority_mutants_y2_end_down = data_norm_ok_y2[data_norm_ok_y2['mutated_genes_light_regime_count'] >= 2]
priority_mutants_y2_end_down_filtered = priority_mutants_y2_end_down[priority_mutants_y2_end_down['outlier_divergent_end_down_y2_count'] >= 2]

priority_mutants_y2_end_up = data_norm_ok_y2[data_norm_ok_y2['mutated_genes_light_regime_count'] >= 2]
priority_mutants_y2_end_up_filtered = priority_mutants_y2_end_up[priority_mutants_y2_end_up['outlier_divergent_end_up_y2_count'] >= 2]

priority_mutants_y2_beg_up = data_norm_ok_y2[data_norm_ok_y2['mutated_genes_light_regime_count'] >= 2]
priority_mutants_y2_beg_up_filtered = priority_mutants_y2_beg_up[priority_mutants_y2_beg_up['outlier_divergent_beg_up_y2_count'] >= 2]

priority_mutants_y2_beg_down = data_norm_ok_y2[data_norm_ok_y2['mutated_genes_light_regime_count'] >= 2]
priority_mutants_y2_beg_down_filtered = priority_mutants_y2_beg_down[priority_mutants_y2_beg_down['outlier_divergent_beg_down_y2_count'] >= 2]

In [8]:
good_mutants_end_down = priority_mutants_y2_end_down_filtered[priority_mutants_y2_end_down_filtered['outlier_divergent_end_down_y2'] == True]
good_mutants_end_up = priority_mutants_y2_end_up_filtered[priority_mutants_y2_end_up_filtered['outlier_divergent_end_up_y2'] == True]
good_mutants_beg_up = priority_mutants_y2_beg_up_filtered[priority_mutants_y2_beg_up_filtered['outlier_divergent_beg_up_y2'] == True]
good_mutants_beg_down = priority_mutants_y2_beg_down_filtered[priority_mutants_y2_beg_down_filtered['outlier_divergent_beg_down_y2'] == True]

In [9]:
priority_mutants_y2_filtered['final_mutated_genes_light_regime_count'] = priority_mutants_y2_filtered.groupby(['mutated_genes', 'light_regime'])['mutant_ID'].transform('count')
good_mutants_end_down['final_mutated_genes_light_regime_count'] = good_mutants_end_down.groupby(['mutated_genes', 'light_regime'])['mutant_ID'].transform('count')
good_mutants_end_up['final_mutated_genes_light_regime_count'] = good_mutants_end_up.groupby(['mutated_genes', 'light_regime'])['mutant_ID'].transform('count')
good_mutants_beg_up['final_mutated_genes_light_regime_count'] = good_mutants_beg_up.groupby(['mutated_genes', 'light_regime'])['mutant_ID'].transform('count')
good_mutants_beg_down['final_mutated_genes_light_regime_count'] = good_mutants_beg_down.groupby(['mutated_genes', 'light_regime'])['mutant_ID'].transform('count')

priority_mutants_y2_filtered['mutant_ratio'] = priority_mutants_y2_filtered['final_mutated_genes_light_regime_count']/priority_mutants_y2_filtered['mutated_genes_light_regime_count']
good_mutants_end_down['mutant_ratio'] = good_mutants_end_down['final_mutated_genes_light_regime_count']/good_mutants_end_down['mutated_genes_light_regime_count']
good_mutants_end_up['mutant_ratio'] = good_mutants_end_up['final_mutated_genes_light_regime_count']/good_mutants_end_up['mutated_genes_light_regime_count']
good_mutants_beg_up['mutant_ratio'] = good_mutants_beg_up['final_mutated_genes_light_regime_count']/good_mutants_beg_up['mutated_genes_light_regime_count']
good_mutants_beg_down['mutant_ratio'] = good_mutants_beg_down['final_mutated_genes_light_regime_count']/good_mutants_beg_down['mutated_genes_light_regime_count']

priority_mutants_y2_filtered_sorted = priority_mutants_y2_filtered.sort_values(by='mean_reduced_v2_y2')
good_mutants_end_down_sorted = good_mutants_end_down.sort_values(by='sum_end_down_y2', ascending=False)
good_mutants_end_up_sorted = good_mutants_end_up.sort_values(by='sum_end_up_y2', ascending=False)
good_mutants_beg_up_sorted = good_mutants_beg_up.sort_values(by='sum_beg_up_y2', ascending=False)
good_mutants_beg_down_sorted = good_mutants_beg_down.sort_values(by='sum_beg_down_y2', ascending=False)

final_list_y2_specific = priority_mutants_y2_filtered_sorted[priority_mutants_y2_filtered_sorted['mutant_ratio'] > 0.65]
final_mutants_end_down = good_mutants_end_down_sorted[good_mutants_end_down_sorted['mutant_ratio'] > 0.65]
final_mutants_end_up = good_mutants_end_up_sorted[good_mutants_end_up_sorted['mutant_ratio'] > 0.65]
final_mutants_beg_up = good_mutants_beg_up_sorted[good_mutants_beg_up_sorted['mutant_ratio'] > 0.65]
final_mutants_beg_down = good_mutants_beg_down_sorted[good_mutants_beg_down_sorted['mutant_ratio'] > 0.65]

/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/996380385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_mutants_y2_filtered['final_mutated_genes_light_regime_count'] = priority_mutants_y2_filtered.groupby(['mutated_genes', 'light_regime'])['mutant_ID'].transform('count')
/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/996380385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_mutants_end_down['final_mutated_genes_light_regime_count'] = good_mutants_end_

In [10]:
final_genes_specific_y2 = final_list_y2_specific['mutated_genes'].unique()
final_genes_end_down_y2 = final_mutants_end_down['mutated_genes'].unique()
final_genes_end_up_y2 = final_mutants_end_up['mutated_genes'].unique()
final_genes_beg_up_y2 = final_mutants_beg_up['mutated_genes'].unique()
final_genes_beg_down_y2 = final_mutants_beg_down['mutated_genes'].unique()

final_list_y2_specific_all_mutants = data_norm_flagged[data_norm_flagged['mutated_genes'].isin(final_genes_specific_y2)]
final_mutants_end_down_all_mutants = data_norm_flagged[data_norm_flagged['mutated_genes'].isin(final_genes_end_down_y2)]
final_mutants_end_up_all_mutants = data_norm_flagged[data_norm_flagged['mutated_genes'].isin(final_genes_end_up_y2)]
final_mutants_beg_up_all_mutants = data_norm_flagged[data_norm_flagged['mutated_genes'].isin(final_genes_beg_up_y2)]
final_mutants_beg_down_all_mutants = data_norm_flagged[data_norm_flagged['mutated_genes'].isin(final_genes_beg_down_y2)]

In [11]:
special_mutants = data_norm_flagged[data_norm_flagged['mutated_genes'] == 'Cre10.g444000']

# Define a function to convert integer to corresponding letter (e.g., 0 -> 'A', 1 -> 'B', ...)
def int_to_letter(num):
    return chr(ord('A') + num)

# Add a 'source_df' column to each DataFrame to identify its source
final_list_y2_specific['source_df'] = 'light_specific_y2'
final_mutants_end_down['source_df'] = 'mutants_end_down_y2'
final_mutants_end_up['source_df'] = 'mutants_end_up_y2'
final_mutants_beg_down['source_df'] = 'mutants_beg_down_y2'
final_mutants_beg_up['source_df'] = 'mutants_beg_up_y2'
special_mutants['source_df'] = 'special_mutants_y2'

final_list_y2_specific_all_mutants['source_df'] = 'light_specific_y2'
final_mutants_end_down_all_mutants['source_df'] = 'mutants_end_down_y2'
final_mutants_end_up_all_mutants['source_df'] = 'mutants_end_up_y2'
final_mutants_beg_down_all_mutants['source_df'] = 'mutants_beg_down_y2'
final_mutants_beg_up_all_mutants['source_df'] = 'mutants_beg_up_y2'

# Concatenate the DataFrames vertically
combined_df = pd.concat([
    final_list_y2_specific,
    final_mutants_end_down,
    final_mutants_end_up,
    final_mutants_beg_down,
    final_mutants_beg_up,
    special_mutants
], ignore_index=True)

combined_df_all_mutants = pd.concat([
    final_list_y2_specific_all_mutants,
    final_mutants_end_down_all_mutants,
    final_mutants_end_up_all_mutants,
    final_mutants_beg_down_all_mutants,
    final_mutants_beg_up_all_mutants,
    special_mutants
], ignore_index=True)

# Convert the 'i' column to corresponding letters
combined_df['i (converted to letter)'] = combined_df['i'].apply(int_to_letter)
combined_df['j'] = combined_df['j'] + 1

combined_df_all_mutants['i (converted to letter)'] = combined_df_all_mutants['i'].apply(int_to_letter)
combined_df_all_mutants['j'] = combined_df_all_mutants['j'] + 1

# Select the desired columns for the final DataFrame
final_df = combined_df
final_df_all_mutants = combined_df_all_mutants
final_df_for_Carolyne = combined_df[['mutant_ID', 'mutated_genes', 'plate', 'i (converted to letter)', 'j', 'source_df']]
final_df_for_Carolyne_all_mutants = combined_df_all_mutants[['mutant_ID', 'mutated_genes', 'plate', 'i (converted to letter)', 'j', 'source_df']]

/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/1668909192.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_list_y2_specific['source_df'] = 'light_specific_y2'
/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/1668909192.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_mutants_end_down['source_df'] = 'mutants_end_down_y2'
/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/1668909192.py:10: SettingWithCopyWarning: 
A value is trying to be set on a 

In [14]:
final_df_for_Carolyne.drop_duplicates(inplace=True)
final_df_for_Carolyne_all_mutants.drop_duplicates(inplace=True)

# keep only the first mutant_ID when there are multiple ones
final_df_for_Carolyne.drop_duplicates(subset=['mutant_ID'], inplace=True)
final_df_for_Carolyne_all_mutants.drop_duplicates(subset=['mutant_ID'], inplace=True)

/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/2971867065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_for_Carolyne.drop_duplicates(inplace=True)
/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/2971867065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_for_Carolyne_all_mutants.drop_duplicates(inplace=True)
/var/folders/1m/j6l58hd9755c0h_89l4yckhw0000gn/T/ipykernel_6327/2971867065.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [15]:
final_df_for_Carolyne

,mutant_ID,mutated_genes,plate,i (converted to letter),j,source_df
0,LMJ.RY0402.052879,Cre16.g675350,20,C,15,light_specific_y2
1,LMJ.RY0402.046388,Cre11.g476050,23,G,8,light_specific_y2
2,LMJ.RY0402.219441,Cre12.g524750,22,L,6,light_specific_y2
3,LMJ.RY0402.135927,Cre12.g538550,21,G,4,light_specific_y2
4,LMJ.RY0402.243547,Cre16.g647850,23,J,5,light_specific_y2
...,...,...,...,...,...,...
1159,LMJ.RY0402.209920,Cre10.g444000,12,C,6,special_mutants_y2
1160,LMJ.RY0402.217896,Cre10.g444000,12,M,8,special_mutants_y2
1167,LMJ.RY0402.252269,Cre10.g444000,15,A,22,special_mutants_y2
1173,LMJ.RY0402.134589,Cre10.g444000,18,D,20,special_mutants_y2


In [16]:
final_df_for_Carolyne_all_mutants

,mutant_ID,mutated_genes,plate,i (converted to letter),j,source_df
0,LMJ.RY0402.062549,Cre09.g394251,2,A,6,light_specific_y2
1,LMJ.RY0402.061906,Cre08.g384150,2,A,23,light_specific_y2
2,LMJ.RY0402.065234,Cre08.g379800,2,B,3,light_specific_y2
3,LMJ.RY0402.068402,Cre10.g437300,2,B,4,light_specific_y2
4,LMJ.RY0402.068880,Cre01.g040250,2,B,15,light_specific_y2
...,...,...,...,...,...,...
3976,LMJ.RY0402.209920,Cre10.g444000,12,C,6,special_mutants_y2
3977,LMJ.RY0402.217896,Cre10.g444000,12,M,8,special_mutants_y2
3984,LMJ.RY0402.252269,Cre10.g444000,15,A,22,special_mutants_y2
3990,LMJ.RY0402.134589,Cre10.g444000,18,D,20,special_mutants_y2
